In [100]:
import json
import fsspec
import re
import posixpath as pp
import pandas as pd 
import sqlite3
import logging
import os

from query_insights.utils import load_config, fs_connection, get_fs_and_abs_path
from query_insights.utils import read_and_process_data, DotifyDict
from query_insights.pre_processing import DBConnection

In [101]:
# change accordingly 
file_names = ["incidents.xlsx"] 

#### creating a config dictionary

In [102]:
data_config = {'path': {
                'data_dictionary_path': '../data/data_dictionary'},
            'db_params': {'db_name': 'sqlite',
                'sqlite_database_path': '../data/db/database.db',
                'chunk_size': 500000},
            'cloud_storage': {'platform': None,
                'prefix_url': None,
                'DefaultEndpointsProtocol': None,
                'account_key_path': None,
                'AccountName': None,
                'EndpointSuffix': None}}

In [103]:
data_config = DotifyDict(data_config)

In [104]:
data_dict_table_path = os.path.join(data_config.path.data_dictionary_path,"raw")
output_json_file_path = data_config.path.data_dictionary_path
cloud_storage = data_config.cloud_storage

#### connection to DB

In [105]:
prefix_url, storage_options = fs_connection(fs_connection_dict = cloud_storage, fs_key= None)

_fs, _ = get_fs_and_abs_path(path=prefix_url, storage_options=storage_options)

database_connection = DBConnection(
            data_config = data_config,
            fs =_fs,
        )

conn = database_connection.connection_db()

for file_name in file_names:
    table_name = file_name.split(".")[0]

    # Connect to the SQLite database
    text_columns = []
    cursor = conn.execute(f"PRAGMA table_info({table_name})")
    for column_info in cursor:
        if column_info[2] == 'TEXT':
            text_columns.append(column_info[1])

### function to create and save a json

In [106]:

def json_file_generator(
    data_dict_table_path,
    file_name,
    output_json_file_path,
    fs=None,
    conn = conn,
    text_columns = text_columns,
    **kwargs,
):
    """
    generate a json file named `<file_name>.json` from a data dictionary csv/xlsx.
    column_name, column_description are expected columns and if the id column is present,
    the output json will have yes to the id key, and id key wont be there for non-id columns.
    If the columns have less than 10 unique values they will be outputted in the json.

    If there are any other columns in the csv/xlsx they will be outputted in the json as it is.
    
    Parameters
    ----------
    data_dict_table_path : str
        input folder path containing the data dictionary (csv/xlsx) file
    file_name : str
        name of the data dictionary (csv/xlsx) file.
    output_json_file_path : str
        output folder path where the JSON files will be saved.
    fs : fsspec.filesystem, optional
        Filesystem of the url, by default ``None``

    Returns
    -------
    None
    """
    # fs = fs or fsspec.filesystem("file")
    result_dict = {}
    df = read_and_process_data(pp.join(data_dict_table_path, file_name))
    
    pattern = re.compile(r"[^\w]")
    
    df['column_name'] = [col.lower().replace(" ", "_") for col in df['column_name']]
    df['column_name'] = [pattern.sub("_", col) for col in df['column_name']]
    
    table_name = file_name.split(".")[0]

    columns_list = []

    for index, row in df.iterrows():
        column_dict = {}
        for column, value in row.items():
            if pd.notna(value):
                if column == "column_name":
                    column_dict["name"] = value
                elif column == "column_description":
                    column_dict["description"] = value
                elif column == "id":
                    column_dict["id"] = "Yes"
                else:
                    
                    column_dict[column] = value
            # text_column and conn are passed here.
            if column in text_columns:    
                unique_values = pd.read_sql(f"SELECT distinct {column} FROM {table_name}", conn).values.ravel().tolist()
                if len(unique_values) < 10:
                    # unique_values = df[col].unique().tolist()
                    column_dict['unique_values'] = unique_values
                
        columns_list.append(column_dict)
        
    result_dict = {
        "table_name": table_name,
        "columns": columns_list
    }
    
    with fs.open(pp.join(output_json_file_path, f"{result_dict['table_name']}.json"), "w") as f:
        json.dump(result_dict, f, indent=4, **kwargs)

In [99]:
for file_name in file_names:
    json_file_generator(data_dict_table_path, file_name, output_json_file_path,_fs, conn, text_columns)